<p style="text-align:center">
    <a href="https://www.ict.mahidol.ac.th/en/" target="_blank">
    <img src="https://www3.ict.mahidol.ac.th/ICTSurveysV2/Content/image/MUICT2.png" width="400" alt="Faculty of ICT">
    </a>
</p>

# Tutorial 12: Computer Vision with Pretrained Models

This tutorial provides a hands-on introduction to computer vision using pre-trained models.  It covers two key tasks: image classification (identifying the main object in an image) and object detection (locating and classifying multiple objects within an image).  Leveraging the power of transfer learning, we'll utilize pre-trained models (like EfficientNet or YOLOv8) to achieve high accuracy without extensive training.  Finally, the tutorial demonstrates how to deploy these computer vision models as interactive web applications using Streamlit, making them easily accessible and shareable.  No prior deep learning expertise is assumed.

### Download the dataset:
1. Download this lab's dataset from here: [LAB12-DATASETS](https://studentmahidolac-my.sharepoint.com/:u:/g/personal/suppawong_tua_mahidol_ac_th/ERH9Og_vxLRLnvuFTxgo8s0BYN7s4pN1tJxuvn8DQch3BA?e=CRwRsN)
2. Extract it under drive D: (to make sure there is no spaces in the path)

In [ ]:
#Define the data path
#It is recommended that you extract "datasets" folder in D:/ to avoid having "white spaces" in the path
data_path = 'lab12_datasets'

## Exercise 01: Image Classification
This tutorial demonstrates image classification using EfficientNet (a pre-trained model) in PyTorch. We'll use a tiny "Cats vs. Dogs" dataset for simplicity.  Topics include model selection,preprocessing, feature extraction, fine-tuning, evaluation, and saving the model.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from PIL import Image  # For checking image loading
import os
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import numpy as np

### 1. Dataset Loading and Preprocessing
We'll use a tiny "Cats vs. Dogs" dataset due to possible limitation of computational resources. The data should be organized into folders: train/cat, train/dog, val/cat, val/dog, test/cat, test/dog.

In [ ]:
data_dir = data_path+'/cats_dogs_tiny' # Replace with your data directory

In [ ]:
# Check if the data directory exists
if not os.path.exists(data_dir):
    print(f"Error: Data directory '{data_dir}' not found. Please download and organize the dataset.")
    exit() # Stop execution if the directory doesn't exist

In [ ]:
# Define data transformations (important for pre-trained models)
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),  # Resize and crop for EfficientNet
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet stats
])

transform_val = transforms.Compose([ # Validation transforms (no random augmentations)
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet stats
])

transform_test = transforms.Compose([ # Test transforms (no random augmentations)
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet stats
])

In [ ]:
# Load the data using ImageFolder (easier with organized folders)
trainset = torchvision.datasets.ImageFolder(root=os.path.join(data_dir, 'train'), transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

valset = torchvision.datasets.ImageFolder(root=os.path.join(data_dir, 'val'), transform=transform_val)
valloader = torch.utils.data.DataLoader(valset, batch_size=32, shuffle=False, num_workers=2)

testset = torchvision.datasets.ImageFolder(root=os.path.join(data_dir, 'test'), transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False, num_workers=2)

classes = trainset.classes # Get the class names directly from the dataset

### 2. Choosing a Pre-trained Model (EfficientNet)
We'll use EfficientNet-b0.  EfficientNet is known for its good performance and efficiency. Other options: ResNet, MobileNet (for mobile).  Consider accuracy vs. speed trade-offs.

In [ ]:
model = torchvision.models.efficientnet_b0(pretrained=True)

### 3. Modifying the Classifier (Fully Connected Layer)

In [ ]:
# EfficientNet's classifier is designed for 1000 ImageNet classes. We need to change it.
num_features = model.classifier[1].in_features  # Access the correct layer for EfficientNet
model.classifier[1] = nn.Linear(num_features, len(classes))  # Replace with our classes

### 4. Training the Model (Fine-tuning)
**Warning**: This step will take a while (~5 minutes) if you run on CPU, even with a tiny training set.

In [ ]:
#Check if CUDA is available
torch.cuda.is_available()

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001) # Adam optimizer often works well

num_epochs = 5  # Adjust as needed

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data in valloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Epoch {epoch+1}, Train Loss: {running_loss/len(trainloader)}, Val Loss: {val_loss/len(valloader)}, Val Accuracy: {100 * correct / total}%")

KeyboardInterrupt: 

### 5. Model Evaluation (on the test set)

In [ ]:
correct = 0
total = 0
y_true = []
y_pred = []

with torch.no_grad():
    for data in testloader:  # Iterate over the validation set
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

print(f"Accuracy of the model on the test images: {100 * correct / total}%")

cm = confusion_matrix(y_true, y_pred)
print("\nConfusion Matrix:")
print(cm)

cr = classification_report(y_true, y_pred, target_names=classes)
print("\nClassification Report:")
print(cr)


### 6. Saving the Model

In [ ]:
torch.save(model.state_dict(), 'cats_dogs_model.pth')

### 7. To load the model (example):

In [ ]:
model = torchvision.models.efficientnet_b0(pretrained=False)
num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_features, len(classes))
model.load_state_dict(torch.load('cats_dogs_model.pth'))
model.eval()

## Exercise 02: Object Detection
This exercise demonstrates object detection using YOLOv8 (a pre-trained model) in PyTorch. We'll use a small publicly available dataset for simplicity.  The focus is on *using* the model, not building it from scratch.

### 1. Install ultralytics (if you haven't already)

In [ ]:
# !pip install ultralytics

In [ ]:
import torch
from ultralytics import YOLO
import cv2  # For image processing
import matplotlib.pyplot as plt  # For displaying images (optional)
import os

### 2. Load a Pre-trained YOLOv8 Model 
YOLOv8 offers different sizes (n, s, m, l, x).  'n' is the smallest and fastest, good for this tutorial. You can choose a larger size if you have more resources and need higher accuracy.

In [ ]:
model = YOLO('yolov8n.pt')  # Load the small YOLOv8 model (pretrained)

### 3. Prepare a Tiny Dataset (Example: Fruits)

For this exercise, let's use a tiny dataset of images with fruits (e.g., apples, bananas). The images should be in a folder (e.g., 'data/fruits_tiny/images').  Annotations (bounding boxes and labels) should be in a corresponding folder (e.g., 'data/fruits_tiny/labels') in YOLO format (txt files).
```
Example directory structure:
fruits_tiny/
  images/
    apple1.jpg
    banana1.jpg
    ...
  labels/
    apple1.txt (contains bounding box coordinates and class label)
    banana1.txt
    ...
```

In [ ]:
#data_dir = data_path+'/fruits_tiny/train'  # Replace with your dataset directory
sample_data_dir = data_path+'/fruits_tiny/sample'

In [ ]:
# Check if the data directory exists
#if not os.path.exists(data_dir):
#    print(f"Error: Data directory '{data_dir}' not found. Please download and organize the dataset.")
#    exit() # Stop execution if the directory doesn't exist

#image_dir = os.path.join(data_dir, 'images')

if not os.path.exists(sample_data_dir):
    print(f"Error: Data directory '{sample_data_dir}' not found. Please download and organize the dataset.")
    exit() # Stop execution if the directory doesn't exist
    
sample_image_dir = os.path.join(sample_data_dir, 'images')

### 4. Perform Object Detection on a Single Image

In [ ]:
image_path = os.path.join(sample_image_dir, '01_ Apple.jpg')  # Example image path
results = model(image_path)  # Run object detection

In [ ]:
# Process the results
for result in results:  # Loop through detected objects
    boxes = result.boxes  # Bounding boxes object
    for box in boxes:
        x1, y1, x2, y2 = box.xyxy[0]  # Bounding box coordinates (top-left, bottom-right)
        cls = int(box.cls[0])  # Class ID
        conf = box.conf[0]  # Confidence score

        # Draw bounding box and label on the image (using OpenCV)
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Correct color format for matplotlib
        cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2) # Green box
        cv2.putText(img, f'{result.names[cls]} {conf:.2f}', (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # (Optional) Display the image with bounding boxes using matplotlib
        plt.imshow(img)
        plt.title(f"Detected Objects in {os.path.basename(image_path)}")
        plt.axis('off')
        plt.show()

### # 5. Object Detection on a Batch of Images

In [ ]:
# You can also perform object detection on a batch of images.

image_files = [os.path.join(sample_image_dir, f) for f in os.listdir(sample_image_dir) if f.endswith(('.jpg', '.png', '.jpeg'))] # Get all image files

results_batch = model(image_files) # Pass a list of images

In [ ]:
# Process the results for each image in the batch (similar to the single image example)
for i, result in enumerate(results_batch):
    boxes = result.boxes  # Bounding boxes object
    for box in boxes:
        x1, y1, x2, y2 = box.xyxy[0]  # Bounding box coordinates (top-left, bottom-right)
        cls = int(box.cls[0])  # Class ID
        conf = box.conf[0]  # Confidence score

        # Draw bounding box and label on the image (using OpenCV)
        img = cv2.imread(image_files[i])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Correct color format for matplotlib
        cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2) # Green box
        cv2.putText(img, f'{result.names[cls]} {conf:.2f}', (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # (Optional) Display the image with bounding boxes using matplotlib
        plt.imshow(img)
        plt.title(f"Detected Objects in {os.path.basename(image_files[i])}")
        plt.axis('off')
        plt.show()

*That is why you should fine-tune the model with your dataset if you can!* However, for this tutorial, we will simply use the pre-trained model without fine-tuning it.

### 6. Model Evaluation (on a validation set)
YOLOv8 provides built-in evaluation metrics.

In [ ]:
# Assuming you have a validation set (e.g., 'fruits_data/val'), you can evaluate the model like this:
results_val = model.val(data=data_path+'/fruits_tiny/fruits_tiny.yaml') # Where fruits_data.yaml is your data configuration file

### 7. Saving the Model

In [ ]:
# The model is already saved when you load it, but if you fine-tune it, you'll want to save the updated weights.
# model.save('fine_tuned_yolov8n.pt')

### 8. Exporting to ONNX (for Streamlit Deployment)
For deployment with Streamlit, exporting to ONNX format is often recommended.

In [ ]:
success = model.export(format='onnx')  # Export the model to ONNX format

## Exercise 03 (Optional): Image Segmentation
This exercise demonstrates image segmentation using a pre-trained DeepLabv3 model in PyTorch. We'll use the PASCAL VOC 2012 dataset (or a subset) for demonstration.  We'll focus on *using* the model for inference, not training or fine-tuning it.

In [ ]:
import torch
import torchvision.transforms as T
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os

### 1. Install Required Libraries (if you haven't already)

In [ ]:
!pip install torchvision segmentation-models-pytorch

### 2. Load the Pre-trained DeepLabv3 Model
We'll use a DeepLabv3 model pre-trained on the PASCAL VOC dataset.  Other options include U-Net, FCN, etc. Segmentation-models-pytorch is an excellent library for this.

In [ ]:
import segmentation_models_pytorch as smp

ENCODER = 'efficientnet-b7' # You can try other encoders like 'resnet50', 'efficientnet-b0', etc.
ENCODER_WEIGHTS = 'imagenet' # Or None if you want to train from scratch
CLASSES = 21 # Number of classes in PASCAL VOC (20 foreground + 1 background)

model = smp.DeepLabV3(
    encoder_name=ENCODER,
    encoder_weights=ENCODER_WEIGHTS,
    classes=CLASSES,
)

model.eval() # Set the model to evaluation mode (important for inference)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Use GPU if available
model.to(device)

### 3. Prepare the Image and Preprocessing

In [ ]:
# Load an image
image_path = data_path+'/seg_example/boat.jpg'  
if not os.path.exists(image_path):
    print(f"Error: Image path '{image_path}' not found.")
    exit()
    
img = Image.open(image_path).convert("RGB") # Open and convert to RGB

In [ ]:
# Define transformations (important for pre-trained models)
# These should match how the model was trained
transform = T.Compose([
    T.Resize([256,344]),  # Resize the image
    T.ToTensor(),  # Convert to tensor
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # Normalize using ImageNet stats
])

input_tensor = transform(img).unsqueeze(0)  # Add a batch dimension (models expect batches)
input_tensor = input_tensor.to(device) # Move tensor to device

### 4. Make Predictions (Inference)

In [ ]:
with torch.no_grad():  # Disable gradient calculations for inference
    output = model(input_tensor)

### 5. Post-processing and Visualization
The output is a tensor containing the segmentation mask (pixel-wise class predictions). Post-processing is usually needed to get a nice segmented image.

In [ ]:
# Get the predicted segmentation mask (the class with the highest probability for each pixel)
mask = torch.argmax(output, dim=1).squeeze().cpu().numpy() # Get the predicted mask

In [ ]:
# Convert the mask to a color image (optional, for better visualization)
# You'll need a colormap that maps class indices to colors.
# Here's a simple example for PASCAL VOC:
VOC_COLORMAP = np.array([[0, 0, 0], [128, 0, 0], [0, 128, 0], [128, 128, 0],
                        [0, 0, 128], [128, 0, 128], [0, 128, 128], [128, 128, 128],
                        [64, 0, 0], [192, 0, 0], [64, 128, 0], [192, 128, 0],
                        [64, 0, 128], [192, 0, 128], [64, 128, 128], [192, 128, 128],
                        [0, 64, 0], [128, 64, 0], [0, 192, 0], [128, 192, 0],
                        [0, 64, 128]])

In [ ]:
segmented_image = VOC_COLORMAP[mask].astype(np.uint8) # Create the segmented image from the mask

In [ ]:
# Display the original and segmented images
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(img)
plt.title("Original Image")

plt.subplot(1, 2, 2)
plt.imshow(segmented_image)
plt.title("Segmented Image")

plt.show()

### 6. Saving the Model
It's best practice to save only the model's state dictionary (the learned weights). This makes the saved file smaller and more portable.

In [ ]:
torch.save(model.state_dict(), 'deeplabv3_model.pth')

In [ ]:
# To load the model in Streamlit:
# model = smp.DeepLabv3(encoder_name=ENCODER, encoder_weights=None, classes=CLASSES) # Create model instance
# model.load_state_dict(torch.load('deeplabv3_model.pth')) # Load saved weights
# model.eval() # Set to evaluation mode

<p style="text-align:center;">That's it! Congratulations! <br> 
    Let's now work on your lab assigment.</p>